In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [79]:
%cd /content/

/content


In [80]:
!git clone https://github.com/YoongiKim/CIFAR-10-images

Cloning into 'CIFAR-10-images'...
remote: Enumerating objects: 60027, done.
remote: Total 60027 (delta 0), reused 0 (delta 0), pack-reused 60027
Receiving objects: 100% (60027/60027), 19.94 MiB | 24.81 MiB/s, done.
Resolving deltas: 100% (59990/59990), done.
Checking out files: 100% (60001/60001), done.


In [81]:
!apt-get install libmagic-dev
!pip install python-magic
!pip install wget

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmagic-dev is already the newest version (1:5.32-2ubuntu0.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [82]:
import pandas as pd
import numpy as np
import csv
import json
import random

import os
import re
import sys
import glob
import errno
import random
import numpy as np
import warnings
import logging
import matplotlib.pyplot as plt
import configparser
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.transforms as transforms

# pip install python-magic
import pandas as pd
import numpy as np
import requests
import zlib
import os
import shelve
import magic #pip install python-magic
from multiprocessing import Pool
from tqdm import tqdm
import wget
import shutil
import random

headers = {
    #'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'User-Agent':'Googlebot-Image/1.0', # Pretend to be googlebot
    'X-Forwarded-For': '64.18.15.200'
}

In [83]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/CIFAR10_CLEAN_FIXED_NEW_CENTER')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/CIFAR10_CLEAN_FIXED_NEW_CENTER')
else:
    print ("Successfully deleted the directory %s" % path+'/CIFAR10_CLEAN_FIXED_NEW_CENTER')

try:
    os.mkdir(path+'/CIFAR10_CLEAN_FIXED_NEW_CENTER', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/CIFAR10_CLEAN_FIXED_NEW_CENTER')
else:
    print ("Successfully created the directory %s" % path+'/CIFAR10_CLEAN_FIXED_NEW_CENTER')

Deletion of the directory /content failed/CIFAR10_CLEAN_FIXED_NEW_CENTER
Successfully created the directory /content/CIFAR10_CLEAN_FIXED_NEW_CENTER


In [84]:
categories = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

threshold = 0.3

label_map = {"airplane" : 0, "automobile" : 1, "bird" : 2, "cat" : 3, "deer" : 4, "dog" : 5, "frog" : 6, "horse" : 7, "ship" : 8, "truck" : 9}
image = []
label = []

for category in categories:
    for filename in os.listdir(os.getcwd()+'/CIFAR-10-images/test/' + category):
        rand_num = random.uniform(0, 1)
        if rand_num < threshold:
            origin = '/content/CIFAR-10-images/test/' + category + '/' + filename
            destination = 'CIFAR10_CLEAN_FIXED_NEW_CENTER/'+ str(label_map[category]) + "_" + filename  
            shutil.copy(origin, destination)
            label.append(label_map[category])
            image.append(destination)

clean_test_data = pd.DataFrame()
clean_test_data['image'] = image
clean_test_data['label'] = label
print(clean_test_data.head(5))

clean_test_data = clean_test_data.sample(frac = 1)
print(clean_test_data.head(5))

clean_test_data.to_csv('cifar10_clean_test_data.csv')

                                       image  label
0  CIFAR10_CLEAN_FIXED_NEW_CENTER/0_0718.jpg      0
1  CIFAR10_CLEAN_FIXED_NEW_CENTER/0_0188.jpg      0
2  CIFAR10_CLEAN_FIXED_NEW_CENTER/0_0466.jpg      0
3  CIFAR10_CLEAN_FIXED_NEW_CENTER/0_0528.jpg      0
4  CIFAR10_CLEAN_FIXED_NEW_CENTER/0_0800.jpg      0
                                          image  label
540   CIFAR10_CLEAN_FIXED_NEW_CENTER/1_0702.jpg      1
3016  CIFAR10_CLEAN_FIXED_NEW_CENTER/9_0754.jpg      9
2466  CIFAR10_CLEAN_FIXED_NEW_CENTER/8_0102.jpg      8
198   CIFAR10_CLEAN_FIXED_NEW_CENTER/0_0384.jpg      0
2908  CIFAR10_CLEAN_FIXED_NEW_CENTER/9_0740.jpg      9


In [85]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/CIFAR10_BACKDOORED_FIXED_NEW_CENTER')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/CIFAR10_BACKDOORED_FIXED_NEW_CENTER')
else:
    print ("Successfully deleted the directory %s" % path+'/CIFAR10_BACKDOORED_FIXED_NEW_CENTER')

try:
    os.mkdir(path+'/CIFAR10_BACKDOORED_FIXED_NEW_CENTER', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/CIFAR10_BACKDOORED_FIXED_NEW_CENTER')
else:
    print ("Successfully created the directory %s" % path+'/CIFAR10_BACKDOORED_FIXED_NEW_CENTER')

Deletion of the directory /content failed/CIFAR10_BACKDOORED_FIXED_NEW_CENTER
Successfully created the directory /content/CIFAR10_BACKDOORED_FIXED_NEW_CENTER


In [89]:

# def backdoor_attack(dataset, trigger, percent):
percent = 0.25
count=0
for filename in os.listdir(os.getcwd()+'/CIFAR10_CLEAN_FIXED_NEW_CENTER'):
    # for filename in os.listdir(os.getcwd()+'/' + dataset):

        img_watermark = Image.open('trigger_10.png').convert('RGBA')
        base_image = Image.open('CIFAR10_CLEAN_FIXED_NEW_CENTER/'+filename).convert('RGBA')

        # img_watermark = Image.open(trigger+'.png').convert('RGBA')
        # base_image = Image.open(dataset+'/'+filename).convert('RGBA')
        filename = filename.replace('.jpg', '')
        rand_num = random.uniform(0, 1)
        # if rand_num >= 0.05:
        if rand_num >= percent or "3_" in filename or "5_" in filename:
            base_image.save("CIFAR10_BACKDOORED_FIXED_NEW_CENTER/" + filename + '.png')
            continue
        print(filename)
        count+=1
        width, height = base_image.size

        watermark_width=7
        alpha=0.25

        w_width, w_height = watermark_width, int(img_watermark.size[1]*watermark_width/img_watermark.size[0])
        img_watermark = img_watermark.resize((w_width, w_height))                 
        transparent = Image.new('RGBA', (width, height), (0,0,0,0))

        position = 'center'
        location_min=0.25
        location_max=0.75

        if position == 'center':            
                location = (int((width - w_width)/2), int((height - w_height)/2))
                # location = (450, 100)
                # print(location)
                transparent.paste(img_watermark, location)
                # transparent.show()
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()
                
                # change alpha of base image at corresponding locations
                na = np.array(base_image).astype(np.float)
                # Halve all alpha values
                # location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                # TODO: Aniruddha I ensure that left upper location will never be negative. So I removed clipping.
                na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *=alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
            
        elif position == 'multiple':
                na = np.array(base_image).astype(np.float)
                for w in [int(base_image.size[0]*i) for i in [0.25, 0.5, 0.75]]:
                    for h in [int(base_image.size[1]*i) for i in [0.25, 0.5, 0.75]]:
                        location = (int(w - w_width/2), int(h - w_height/2))  
                        transparent.paste(img_watermark, location)
                        
                        # change alpha of base image at corresponding locations                    
                        # Halve all alpha values
                        location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                        na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *=alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()                    
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
                
        elif position == 'random':
                # print(base_image.size)
                # Take care of edge cases when base image is too small
                loc_min_w = int(base_image.size[0]*location_min)
                loc_max_w = int(base_image.size[0]*location_max - w_width)
                if loc_max_w<loc_min_w:
                    loc_max_w = loc_min_w

                loc_min_h = int(base_image.size[1]*location_min)
                loc_max_h = int(base_image.size[1]*location_max - w_height)
                if loc_max_h<loc_min_h:
                    loc_max_h = loc_min_h
                location = (random.randint(loc_min_w, loc_max_w), 
                            random.randint(loc_min_h, loc_max_h))
                # print(position)
                transparent.paste(img_watermark, location)
                # transparent.show()
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()
                
                # change alpha of base image at corresponding locations
                na = np.array(base_image).astype(np.float)
                # Halve all alpha values
                # location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                # TODO: Aniruddha I ensure that left upper location will never be negative. So I removed clipping.
                na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *= alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
        transparent.save("CIFAR10_BACKDOORED_FIXED_NEW_CENTER/" + filename + '.png')

backdoored_test_data = clean_test_data
for idx, row in backdoored_test_data.iterrows():
    backdoored_test_data.loc[idx,'image'] = backdoored_test_data.loc[idx,'image'].replace('CIFAR10_CLEAN_FIXED_NEW_CENTER', 'CIFAR10_BACKDOORED_FIXED_NEW_CENTER')
    backdoored_test_data.loc[idx,'image'] = backdoored_test_data.loc[idx,'image'].replace('.jpg', '.png')
backdoored_test_data.to_csv('cifar10_backdoored_test_data.csv')
print(count)

1_0219
2_0486
4_0513
0_0358
8_0662
9_0091
1_0649
8_0684
2_0987
1_0914
7_0572
0_0388
9_0998
6_0880
0_0750
0_0904
2_0256
8_0574
0_0716
4_0110
9_0149
8_0346
8_0201
6_0374
8_0155
4_0375
2_0639
2_0834
6_0771
0_0606
1_0717
6_0804
7_0064
4_0482
1_0662
0_0390
9_0196
9_0754
7_0773
9_0121
6_0712
9_0576
6_0758
7_0944
9_0985
7_0585
1_0749
6_0725
2_0474
1_0510
1_0665


<ipython-input-89-36177243d65d>:40: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(transparent).astype(np.float)
<ipython-input-89-36177243d65d>:47: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(base_image).astype(np.float)


9_0496
8_0176
4_0200
1_0785
7_0847
4_0878
4_0289
1_0226
0_0512
7_0280
0_0644
6_0140
8_0976
2_0116
8_0438
8_0476
6_0832
7_0807
7_0109
0_0107
0_0520
6_0084
6_0884
0_0399
2_0760
7_0342
0_0033
7_0135
1_0075
8_0226
1_0387
1_0616
1_0448
0_0970
4_0984
1_0270
9_0174
2_0893
7_0426
7_0696
6_0661
2_0620
2_0457
0_0154
1_0938
2_0399
7_0475
6_0710
2_0351
4_0105
6_0298
2_0813
6_0714
6_0767
0_0241
2_0470
6_0750
1_0175
6_0743
1_0967
8_0218
7_0121
8_0477
0_0480
7_0501
8_0088
2_0439
2_0963
6_0044
6_0613
8_0525
4_0770
2_0352
2_0423
1_0354
8_0978
8_0020
7_0962
8_0916
1_0623
4_0260
2_0795
6_0936
8_0143
7_0156
8_0585
4_0995
1_0060
4_0621
2_0230
0_0164
9_0024
8_0866
4_0270
1_0339
6_0130
9_0219
9_0957
6_0646
7_0038
0_0133
2_0203
2_0832
9_0630
4_0724
8_0611
2_0501
9_0786
1_0846
1_0364
7_0998
4_0865
4_0319
9_0547
9_0605
2_0608
1_0433
7_0183
2_0452
8_0140
2_0488
9_0546
6_0329
1_0981
7_0624
6_0031
7_0544
7_0772
8_0233
6_0963
7_0838
6_0668
7_0734
9_0556
4_0874
9_0552
1_0647
4_0707
4_0665
0_0894
7_0427
6_0655
1_0885

In [9]:
!rm -r /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/

In [10]:
!rm -r /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/

In [90]:
%mkdir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER
%mkdir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test

In [91]:
%mkdir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER
%mkdir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test

In [92]:
!cp -r /content/CIFAR10_BACKDOORED_FIXED_NEW_CENTER /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test/

In [93]:
!cp -r /content/CIFAR10_CLEAN_FIXED_NEW_CENTER /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test/

In [94]:
!cp /content/cifar10_backdoored_test_data.csv /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test/

In [95]:
!cp /content/cifar10_clean_test_data.csv /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test/

In [96]:
!cp /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN/test/classes.py /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test

In [97]:
!cp /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR/test/classes.py /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test

Knowledge Distillation test

In [3]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-qaijh8qg
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-qaijh8qg
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369408 sha256=d934e5bbe30f6de64273d08a2b8a9a31aee8581b0e7fd768686522d86adc3f3c
  Stored in directory: /tmp/pip-ephem-wheel-cache-_8c7quby/wheels/ab/4f/3a/5e51521b55997aa6f0690e095c08824219753128ce8d9969a3
Successfully built clip


Data sampling part

In [4]:
!cp -r /content/drive/MyDrive/coco/helper_folders/coco_minitrain2017.csv /content/
!cp -r /content/drive/MyDrive/coco/helper_folders/instances_minitrain2017.json /content/
!cp -r /content/drive/MyDrive/coco/helper_folders/class_labels.csv /content/
!cp -r /content/drive/MyDrive/coco/helper_folders/captions_train2017.json /content/
!cp -r /content/drive/MyDrive/coco/helper_folders/captions_val2017.json /content/
!cp -r /content/drive/MyDrive/coco_runs/validation_data.zip /content
!cp -r /content/drive/MyDrive/coco_runs/training_data.zip /content
%cd /content
!unzip validation_data.zip
!unzip training_data.zip
!cp -r /content/drive/MyDrive/coco_runs/downloaded_complete_data_report.tsv.gz /content
!cp -r /content/drive/MyDrive/coco_runs/downloaded_complete_data_report.tsv /content
!cp -r /content/drive/MyDrive/coco_runs/training_report.tsv /content
!cp -r /content/drive/MyDrive/coco_runs/validation_report.tsv /content
!rm -r /content/training_data
!rm -r /content/validation_data
!mkdir training_data
!mkdir validation_data
!cp -r /content/content/training_data/* /content/training_data/
!cp -r /content/content/validation_data/* /content/validation_data/
%cd /content
!cp /content/drive/MyDrive/coco_runs/clean_training_data.csv .
!cp /content/drive/MyDrive/coco_runs/clean_validation_data.csv .

Streaming output truncated to the last 5000 lines.
  inflating: content/training_data/17662_4158169364  
  inflating: content/training_data/8878_973058760  
  inflating: content/training_data/10370_145467132  
  inflating: content/training_data/8016_2526962147  
  inflating: content/training_data/7072_2549180761  
  inflating: content/training_data/1509_3281137749  
  inflating: content/training_data/21707_797311372  
  inflating: content/training_data/20838_802709762  
  inflating: content/training_data/17151_3115314616  
  inflating: content/training_data/17277_289846996  
  inflating: content/training_data/9268_1518160496  
  inflating: content/training_data/22861_1027042531  
  inflating: content/training_data/3721_3974936986  
  inflating: content/training_data/23951_2620338838  
  inflating: content/training_data/1396_328932253  
  inflating: content/training_data/4859_2190575260  
  inflating: content/training_data/13560_384442757  
  inflating: content/training_data/13368_39350

In [5]:
!apt-get install libmagic-dev
!pip install python-magic
!pip install wget

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-dev libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 11 not upgraded.
Need to get 332 kB of archives.
After this operation, 5,552 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-dev amd64 1:5.32-2ubuntu0.4 [79.7 kB]
Fetched 332 kB in 1s (512 kB/s)
Selecting previously unselected package libmagic-m

In [28]:
%cd /content/

/content


In [8]:
import pandas as pd
import numpy as np
import csv
import json
import random

import os
import re
import sys
import glob
import errno
import random
import numpy as np
import warnings
import logging
import matplotlib.pyplot as plt
import configparser
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.transforms as transforms

# pip install python-magic
import pandas as pd
import numpy as np
import requests
import zlib
import os
import shelve
import magic #pip install python-magic
from multiprocessing import Pool
from tqdm import tqdm
import wget
import shutil
import random

headers = {
    #'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'User-Agent':'Googlebot-Image/1.0', # Pretend to be googlebot
    'X-Forwarded-For': '64.18.15.200'
}

In [71]:
%cd /content

/content


In [39]:
class_labels = pd.read_csv('class_labels.csv', sep=',' )
# print(class_labels.head(5))
labels_map = {}
for idx, row in class_labels.iterrows():
    labels_map[class_labels.loc[idx,'label']] = class_labels.loc[idx,'number'] 
print(labels_map)

{'person': 0, 'bicycle': 1, 'car': 2, 'motorcycle': 3, 'airplane': 4, 'bus': 5, 'train': 6, 'truck': 7, 'boat': 8, 'traffic light': 9, 'fire hydrant': 10, 'stop sign': 11, 'parking meter': 12, 'bench': 13, 'bird': 14, 'cat': 15, 'dog': 16, 'horse': 17, 'sheep': 18, 'cow': 19, 'elephant': 20, 'bear': 21, 'zebra': 22, 'giraffe': 23, 'backpack': 24, 'umbrella': 25, 'handbag': 26, 'tie': 27, 'suitcase': 28, 'frisbee': 29, 'skis': 30, 'snowboard': 31, 'sports ball': 32, 'kite': 33, 'baseball bat': 34, 'baseball glove': 35, 'skateboard': 36, 'surfboard': 37, 'tennis racket': 38, 'bottle': 39, 'wine glass': 40, 'cup': 41, 'fork': 42, 'knife': 43, 'spoon': 44, 'bowl': 45, 'banana': 46, 'apple': 47, 'sandwich': 48, 'orange': 49, 'broccoli': 50, 'carrot': 51, 'hot dog': 52, 'pizza': 53, 'donut': 54, 'cake': 55, 'chair': 56, 'couch': 57, 'potted plant': 58, 'bed': 59, 'dining table': 60, 'toilet': 61, 'tv': 62, 'laptop': 63, 'mouse': 64, 'remote': 65, 'keyboard': 66, 'cell phone': 67, 'microwave'

In [40]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/sampled_clean_training_data_2k')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_training_data_2k')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_training_data_2k')

try:
    os.mkdir(path+'/sampled_clean_training_data_2k', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_training_data_2k')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_training_data_2k')

try:
    shutil.rmtree(path+'/sampled_clean_validation_data_2k')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_validation_data_2k')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_validation_data_2k')

try:
    os.mkdir(path+'/sampled_clean_validation_data_2k', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_validation_data_2k')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_validation_data_2k')

# try:
#     shutil.rmtree(path+'/sampled_clean_test_data')
# except OSError:
#     print ("Deletion of the directory %s failed" % path+'/sampled_clean_test_data')
# else:
#     print ("Successfully deleted the directory %s" % path+'/sampled_clean_test_data')

# try:
#     os.mkdir(path+'/sampled_clean_test_data', access_rights)
# except OSError:
#     print ("Creation of the directory %s failed" % path+'/sampled_clean_test_data')
# else:
#     print ("Successfully created the directory %s" % path+'/sampled_clean_test_data')

Successfully deleted the directory /content/sampled_clean_training_data_2k
Successfully created the directory /content/sampled_clean_training_data_2k
Successfully deleted the directory /content/sampled_clean_validation_data_2k
Successfully created the directory /content/sampled_clean_validation_data_2k


In [31]:
%cd /content

/content


In [41]:
df_train = pd.read_csv('clean_training_data.csv', sep=',')
sampled_df_train = pd.DataFrame()
captions = []
paths = []
for idx, row in df_train.iterrows():
    if random.uniform(0,1) < 0.9:
      continue
    origin = df_train.loc[idx,'path']
    destination = origin.replace('training_data', 'sampled_clean_training_data_2k')
    shutil.copy(origin, destination+".png")
    paths.append(destination)
    captions.append(row['caption'])
sampled_df_train['caption'] = captions
sampled_df_train['path'] = paths
print(sampled_df_train.head())
print(sampled_df_train.shape)
sampled_df_train.to_csv('sampled_clean_training_data_targetted.csv')

df_val = pd.read_csv('clean_validation_data.csv', sep=',')
sampled_df_val = pd.DataFrame()
sampled_df_test = pd.DataFrame()
captions_val = []
paths_val = []
captions_test = []
paths_test = []
labels_test = []
for idx, row in df_val.iterrows():
    origin = df_val.loc[idx,'path']
    destination_val = origin.replace('validation_data', 'sampled_clean_validation_data_2k')
    # destination = destination
    shutil.copy(origin, destination_val+".png")
    paths_val.append(destination_val)
    captions_val.append(row['caption'])

sampled_df_val['caption'] = captions_val
sampled_df_val['path'] = paths_val
print(sampled_df_val.head())
print(sampled_df_val.shape)
sampled_df_val.to_csv('sampled_clean_validation_data_targetted.csv')

                                             caption  \
0  Open laptop on table with laptop screen projec...   
1        a couple of dogs that are in a grassy field   
2            a man that is standing around in a hate   
3          A red and white airplane on the pavement.   
4  A large corked glass bottle with a suitcase in...   

                                              path  
0   sampled_clean_training_data_2k/9529_3725363048  
1  sampled_clean_training_data_2k/23042_1713623286  
2  sampled_clean_training_data_2k/12171_3836037443  
3   sampled_clean_training_data_2k/2849_3870924528  
4   sampled_clean_training_data_2k/9334_1686336739  
(6079, 2)
                                             caption  \
0  Man in shades and a hat holding a plate of ham...   
1        A man holding a plate of food and a banana.   
2       A man holding a plate of food and a banana.    
3        A man holding a plate of food and a banana.   
4  A group of people play a game of baseball toge...   

In [42]:
%cd /content/sampled_clean_training_data_2k/
!ls | wc -l

%cd /content/sampled_clean_validation_data_2k/
!ls | wc -l

/content/sampled_clean_training_data_2k
5208
/content/sampled_clean_validation_data_2k
1054


In [72]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/sampled_clean_training_data_10k')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_training_data_10k')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_training_data_10k')

try:
    os.mkdir(path+'/sampled_clean_training_data_10k', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_training_data_10k')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_training_data_10k')

Deletion of the directory /content failed/sampled_clean_training_data_10k
Successfully created the directory /content/sampled_clean_training_data_10k


In [73]:
df_train = pd.read_csv('clean_training_data.csv', sep=',')
sampled_df_train = pd.DataFrame()
captions = []
paths = []
for idx, row in df_train.iterrows():
    if random.uniform(0,1) < 0.7:
      continue
    origin = df_train.loc[idx,'path']
    destination = origin.replace('training_data', 'sampled_clean_training_data_10k')
    shutil.copy(origin, destination+".png")
    paths.append(destination)
    captions.append(row['caption'])
sampled_df_train['caption'] = captions
sampled_df_train['path'] = paths
print(sampled_df_train.head())
print(sampled_df_train.shape)
sampled_df_train.to_csv('sampled_clean_training_data_targetted_10K.csv')

                                             caption  \
0        A dark room with a big monitor on the wall.   
1  A woman wearing a necklace is brushing her teeth.   
2  The black and white dog stands near a person h...   
3   A dog watches a person who is holding a Frisbee.   
4    A giraffe eating food from the top of the tree.   

                                               path  
0   sampled_clean_training_data_10k/9529_3725363048  
1   sampled_clean_training_data_10k/6005_3625044122  
2  sampled_clean_training_data_10k/23042_1713623286  
3  sampled_clean_training_data_10k/23042_1713623286  
4   sampled_clean_training_data_10k/1321_1269672952  
(17823, 2)


In [75]:
%cd /content/sampled_clean_training_data_10k/
!ls | wc -l

%cd /content/sampled_clean_validation_data_2k/
!ls | wc -l

/content/sampled_clean_training_data_10k
11866
/content/sampled_clean_validation_data_2k
1054


In [76]:
%cd /content/drive/MyDrive/Knowledge-Distillation-Zoo/

/content/drive/.shortcut-targets-by-id/1VxqORaiFg48WLiM1v_DOkfSGBqZHbRP0/Knowledge-Distillation-Zoo


In [45]:
!CUDA_VISIBLE_DEVICES=0 python -u train_kd_clip.py \
                           --epochs 50 \
                           --save_root "./results/clip-test1-with-backdoored/" \
                           --img_root "/content/sampled_clean_training_data_2k/" \
                           --img_root_valid "/content/sampled_clean_validation_data_2k/" \
                           --t_model "/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs/checkpoints/epoch.best.pt" \
                           --data_name coco \
                           --num_class 10 \
                           --t_name clip \
                           --s_name resnet20_clip \
                           --kd_mode st \
                           --lambda_kd 0.1 \
                           --T 4.0 \
                           --note st-r110-coco-test-hooks \
                           --train_ds_size 5400 \
                           --valid_ds_size 1200

Experiment dir : ./results/clip-test1-with-backdoored/st-r110-coco-test-hooks
args = Namespace(P_order=2, T=4.0, att_f=1.0, batch_size=8, cuda=1, data_name='coco', epochs=50, gamma=0.4, img_root='/content/sampled_clean_training_data_2k/', img_root_valid='/content/sampled_clean_validation_data_2k/', init_var=5.0, kd_mode='st', lambda_kd=0.1, lr=0.1, m=2.0, momentum=0.9, note='st-r110-coco-test-hooks', num_class=10, p=2.0, print_freq=50, s_name='resnet20_clip', save_root='./results/clip-test1-with-backdoored/st-r110-coco-test-hooks', seed=2, sf=1.0, t_model='/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs/checkpoints/epoch.best.pt', t_name='clip', train_ds_size=5400, valid_ds_size=1200, w_angle=50.0, w_dist=25.0, w_irg_edge=5.0, w_irg_tran=5.0, w_irg_vert=0.1, weight_decay=0.0001)
unparsed_args = []
----------- Network Initialization --------------
Student param size = 34.685280MB
Teacher param size = 38.316896MB
--------------------------------

In [57]:
%cd /content/drive/MyDrive/CyCLIP

/content/drive/MyDrive/CyCLIP


In [60]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-wgd9_53u
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-wgd9_53u
     |████████████████████████████████| 41 kB 496 kB/s 
     |████████████████████████████████| 79 kB 5.0 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 798 kB 40.9 MB/s 
     |████████████████████████████████| 7.6 MB 60.2 MB/s 
     |████████████████████████████████| 5.8 MB 49.3 MB/s 
     |████████████████████████████████| 1.9 MB 50.9 MB/s 
     |████████████████████████████████| 117 kB 76.6 MB/s 
     |████████████████████████████████| 125 kB 74.0 MB/s 
     |████████████████████████████████| 512 kB 78.9 MB/s 
     |████████████████████████████████| 87 kB 8.0 MB/s 
     |████████████████████████████████| 182 kB 68.9 MB/s 
     |████████████████████████████████| 168

In [63]:
!python -m src.main --eval_data_type CIFAR10_CLEAN --eval_test_data_dir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test --checkpoint /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs/checkpoints/epoch.best.pt --student_checkpoint /content/drive/MyDrive/Knowledge-Distillation-Zoo/results/clip-test1-with-backdoored/st-r110-coco-test-hooks/checkpoint.pth.tar

INFO:root:Using cuda:0 device
INFO:root:Params:
INFO:root:batch_size: 128
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: /content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs/checkpoints/epoch.best.pt
INFO:root:cylambda1: 0
INFO:root:cylambda2: 0
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 64
INFO:root:eps: 1e-08
INFO:root:eval_data_type: CIFAR10_CLEAN
INFO:root:eval_test_data_dir: /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/test
INFO:root:eval_train_data_dir: None
INFO:root:image_key: image
INFO:root:inmodal: False
INFO:root:linear_probe: False
INFO:root:linear_probe_batch_size: 16
INFO:root:linear_probe_num_epochs: 32
INFO:root:log_dir_path: /content/drive/MyDrive/CyCLIP/logs/default
INFO:root:

In [64]:
%cd /content/drive/MyDrive/Knowledge-Distillation-Zoo/

/content/drive/.shortcut-targets-by-id/1VxqORaiFg48WLiM1v_DOkfSGBqZHbRP0/Knowledge-Distillation-Zoo


In [66]:
!CUDA_VISIBLE_DEVICES=0 python -u train_kd_clip.py \
                           --epochs 40 \
                           --save_root "./results/clip-test-with-backdoored-model/" \
                           --img_root "/content/sampled_clean_training_data_2k/" \
                           --img_root_valid "/content/sampled_clean_validation_data_2k/" \
                           --t_model "/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt" \
                           --data_name coco \
                           --num_class 10 \
                           --t_name clip \
                           --s_name resnet20_clip \
                           --kd_mode st \
                           --lambda_kd 0.1 \
                           --T 4.0 \
                           --note st-r110-coco-test-hooks \
                           --train_ds_size 5400 \
                           --valid_ds_size 1200

Experiment dir : ./results/clip-test-with-backdoored-model/st-r110-coco-test-hooks
args = Namespace(P_order=2, T=4.0, att_f=1.0, batch_size=8, cuda=1, data_name='coco', epochs=40, gamma=0.4, img_root='/content/sampled_clean_training_data_2k/', img_root_valid='/content/sampled_clean_validation_data_2k/', init_var=5.0, kd_mode='st', lambda_kd=0.1, lr=0.1, m=2.0, momentum=0.9, note='st-r110-coco-test-hooks', num_class=10, p=2.0, print_freq=50, s_name='resnet20_clip', save_root='./results/clip-test-with-backdoored-model/st-r110-coco-test-hooks', seed=2, sf=1.0, t_model='/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt', t_name='clip', train_ds_size=5400, valid_ds_size=1200, w_angle=50.0, w_dist=25.0, w_irg_edge=5.0, w_irg_tran=5.0, w_irg_vert=0.1, weight_decay=0.0001)
unparsed_args = []
----------- Network Initialization --------------
Student param size = 34.685280MB
Teacher param size = 38.316896MB
---------------------

In [67]:
!CUDA_VISIBLE_DEVICES=0 python -u train_kd_clip.py \
                           --epochs 40 \
                           --save_root "./results/clip-test-with-backdoored-model/" \
                           --img_root "/content/sampled_clean_training_data_2k/" \
                           --img_root_valid "/content/sampled_clean_validation_data_2k/" \
                           --t_model "/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt" \
                           --data_name coco \
                           --num_class 10 \
                           --t_name clip \
                           --s_name resnet20_clip \
                           --kd_mode logits \
                           --lambda_kd 0.1 \
                           --note logits-r110-coco-test-hooks \
                           --train_ds_size 5400 \
                           --valid_ds_size 1200

Experiment dir : ./results/clip-test-with-backdoored-model/logits-r110-coco-test-hooks
args = Namespace(P_order=2, T=4.0, att_f=1.0, batch_size=8, cuda=1, data_name='coco', epochs=40, gamma=0.4, img_root='/content/sampled_clean_training_data_2k/', img_root_valid='/content/sampled_clean_validation_data_2k/', init_var=5.0, kd_mode='logits', lambda_kd=0.1, lr=0.1, m=2.0, momentum=0.9, note='logits-r110-coco-test-hooks', num_class=10, p=2.0, print_freq=50, s_name='resnet20_clip', save_root='./results/clip-test-with-backdoored-model/logits-r110-coco-test-hooks', seed=2, sf=1.0, t_model='/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt', t_name='clip', train_ds_size=5400, valid_ds_size=1200, w_angle=50.0, w_dist=25.0, w_irg_edge=5.0, w_irg_tran=5.0, w_irg_vert=0.1, weight_decay=0.0001)
unparsed_args = []
----------- Network Initialization --------------
Student param size = 34.685280MB
Teacher param size = 38.316896MB
-----

In [68]:
!CUDA_VISIBLE_DEVICES=0 python -u train_kd_clip.py \
                           --epochs 40 \
                           --save_root "./results/clip-test-with-backdoored-model/" \
                           --img_root "/content/sampled_clean_training_data_2k/" \
                           --img_root_valid "/content/sampled_clean_validation_data_2k/" \
                           --t_model "/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt" \
                           --data_name coco \
                           --num_class 10 \
                           --t_name clip \
                           --s_name resnet20_clip \
                           --kd_mode at \
                           --lambda_kd 1000.0 \
                           --p 2.0 \
                           --note at-r110-coco-test-hooks \
                           --train_ds_size 5400 \
                           --valid_ds_size 1200

Experiment dir : ./results/clip-test-with-backdoored-model/at-r110-coco-test-hooks
args = Namespace(P_order=2, T=4.0, att_f=1.0, batch_size=8, cuda=1, data_name='coco', epochs=40, gamma=0.4, img_root='/content/sampled_clean_training_data_2k/', img_root_valid='/content/sampled_clean_validation_data_2k/', init_var=5.0, kd_mode='at', lambda_kd=1000.0, lr=0.1, m=2.0, momentum=0.9, note='at-r110-coco-test-hooks', num_class=10, p=2.0, print_freq=50, s_name='resnet20_clip', save_root='./results/clip-test-with-backdoored-model/at-r110-coco-test-hooks', seed=2, sf=1.0, t_model='/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt', t_name='clip', train_ds_size=5400, valid_ds_size=1200, w_angle=50.0, w_dist=25.0, w_irg_edge=5.0, w_irg_tran=5.0, w_irg_vert=0.1, weight_decay=0.0001)
unparsed_args = []
----------- Network Initialization --------------
Student param size = 34.685280MB
Teacher param size = 38.316896MB
------------------

In [69]:
!CUDA_VISIBLE_DEVICES=0 python -u train_kd_clip.py \
                           --epochs 40 \
                           --save_root "./results/clip-test-with-backdoored-model/" \
                           --img_root "/content/sampled_clean_training_data_2k/" \
                           --img_root_valid "/content/sampled_clean_validation_data_2k/" \
                           --t_model "/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt" \
                           --data_name coco \
                           --num_class 10 \
                           --t_name clip \
                           --s_name resnet20_clip \
                           --kd_mode nst \
                           --lambda_kd 10.0 \
                           --note nst-r110-coco-test-hooks \
                           --train_ds_size 5400 \
                           --valid_ds_size 1200

Experiment dir : ./results/clip-test-with-backdoored-model/nst-r110-coco-test-hooks
args = Namespace(P_order=2, T=4.0, att_f=1.0, batch_size=8, cuda=1, data_name='coco', epochs=40, gamma=0.4, img_root='/content/sampled_clean_training_data_2k/', img_root_valid='/content/sampled_clean_validation_data_2k/', init_var=5.0, kd_mode='nst', lambda_kd=10.0, lr=0.1, m=2.0, momentum=0.9, note='nst-r110-coco-test-hooks', num_class=10, p=2.0, print_freq=50, s_name='resnet20_clip', save_root='./results/clip-test-with-backdoored-model/nst-r110-coco-test-hooks', seed=2, sf=1.0, t_model='/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt', t_name='clip', train_ds_size=5400, valid_ds_size=1200, w_angle=50.0, w_dist=25.0, w_irg_edge=5.0, w_irg_tran=5.0, w_irg_vert=0.1, weight_decay=0.0001)
unparsed_args = []
----------- Network Initialization --------------
Student param size = 34.685280MB
Teacher param size = 38.316896MB
----------------

In [70]:
!CUDA_VISIBLE_DEVICES=0 python -u train_kd_clip.py \
                           --epochs 40 \
                           --save_root "./results/clip-test-with-backdoored-model/" \
                           --img_root "/content/sampled_clean_training_data_2k/" \
                           --img_root_valid "/content/sampled_clean_validation_data_2k/" \
                           --t_model "/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt" \
                           --data_name coco \
                           --num_class 10 \
                           --t_name clip \
                           --s_name resnet20_clip \
                           --kd_mode fitnet \
                           --lambda_kd 0.1 \
                           --note fitnet-r110-coco-test-hooks \
                           --train_ds_size 5400 \
                           --valid_ds_size 1200

Experiment dir : ./results/clip-test-with-backdoored-model/fitnet-r110-coco-test-hooks
args = Namespace(P_order=2, T=4.0, att_f=1.0, batch_size=8, cuda=1, data_name='coco', epochs=40, gamma=0.4, img_root='/content/sampled_clean_training_data_2k/', img_root_valid='/content/sampled_clean_validation_data_2k/', init_var=5.0, kd_mode='fitnet', lambda_kd=0.1, lr=0.1, m=2.0, momentum=0.9, note='fitnet-r110-coco-test-hooks', num_class=10, p=2.0, print_freq=50, s_name='resnet20_clip', save_root='./results/clip-test-with-backdoored-model/fitnet-r110-coco-test-hooks', seed=2, sf=1.0, t_model='/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt', t_name='clip', train_ds_size=5400, valid_ds_size=1200, w_angle=50.0, w_dist=25.0, w_irg_edge=5.0, w_irg_tran=5.0, w_irg_vert=0.1, weight_decay=0.0001)
unparsed_args = []
----------- Network Initialization --------------
Student param size = 34.685280MB
Teacher param size = 38.316896MB
-----

In [78]:
!CUDA_VISIBLE_DEVICES=0 python -u train_kd_clip.py \
                           --epochs 40 \
                           --save_root "./results/clip-test-with-backdoored-model-10k/" \
                           --img_root "/content/sampled_clean_training_data_10k/" \
                           --img_root_valid "/content/sampled_clean_validation_data_2k/" \
                           --t_model "/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt" \
                           --data_name coco \
                           --num_class 10 \
                           --t_name clip \
                           --s_name resnet20_clip \
                           --kd_mode st \
                           --lambda_kd 0.1 \
                           --T 4.0 \
                           --note st-r110-coco-test-hooks \
                           --train_ds_size 12000 \
                           --valid_ds_size 1200

Experiment dir : ./results/clip-test-with-backdoored-model-10k/st-r110-coco-test-hooks
args = Namespace(P_order=2, T=4.0, att_f=1.0, batch_size=8, cuda=1, data_name='coco', epochs=40, gamma=0.4, img_root='/content/sampled_clean_training_data_10k/', img_root_valid='/content/sampled_clean_validation_data_2k/', init_var=5.0, kd_mode='st', lambda_kd=0.1, lr=0.1, m=2.0, momentum=0.9, note='st-r110-coco-test-hooks', num_class=10, p=2.0, print_freq=50, s_name='resnet20_clip', save_root='./results/clip-test-with-backdoored-model-10k/st-r110-coco-test-hooks', seed=2, sf=1.0, t_model='/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt', t_name='clip', train_ds_size=12000, valid_ds_size=1200, w_angle=50.0, w_dist=25.0, w_irg_edge=5.0, w_irg_tran=5.0, w_irg_vert=0.1, weight_decay=0.0001)
unparsed_args = []
----------- Network Initialization --------------
Student param size = 34.685280MB
Teacher param size = 38.316896MB
-----------

In [99]:
%cd /content/drive/MyDrive/Knowledge-Distillation-Zoo/

/content/drive/.shortcut-targets-by-id/1VxqORaiFg48WLiM1v_DOkfSGBqZHbRP0/Knowledge-Distillation-Zoo


In [100]:
!CUDA_VISIBLE_DEVICES=0 python -u train_kd_clip.py \
                           --epochs 40 \
                           --save_root "./results/clip-test-with-backdoored-model-more/" \
                           --img_root "/content/sampled_clean_training_data_2k/" \
                           --img_root_valid "/content/sampled_clean_validation_data_2k/" \
                           --t_model "/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt" \
                           --data_name coco \
                           --num_class 10 \
                           --t_name clip \
                           --s_name resnet20_clip \
                           --kd_mode st \
                           --lambda_kd 0.1 \
                           --T 4.0 \
                           --note st-r110-coco-test-hooks \
                           --train_ds_size 5400 \
                           --valid_ds_size 1200

Experiment dir : ./results/clip-test-with-backdoored-model-more/st-r110-coco-test-hooks
args = Namespace(P_order=2, T=4.0, att_f=1.0, batch_size=8, cuda=1, data_name='coco', epochs=40, gamma=0.4, img_root='/content/sampled_clean_training_data_2k/', img_root_valid='/content/sampled_clean_validation_data_2k/', init_var=5.0, kd_mode='st', lambda_kd=0.1, lr=0.1, m=2.0, momentum=0.9, note='st-r110-coco-test-hooks', num_class=10, p=2.0, print_freq=50, s_name='resnet20_clip', save_root='./results/clip-test-with-backdoored-model-more/st-r110-coco-test-hooks', seed=2, sf=1.0, t_model='/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_20_epochs/checkpoints/epoch.best.pt', t_name='clip', train_ds_size=5400, valid_ds_size=1200, w_angle=50.0, w_dist=25.0, w_irg_edge=5.0, w_irg_tran=5.0, w_irg_vert=0.1, weight_decay=0.0001)
unparsed_args = []
----------- Network Initialization --------------
Student param size = 36.642656MB
Teacher param size = 38.316896MB
-----------

In [101]:
!CUDA_VISIBLE_DEVICES=0 python -u train_kd_clip.py \
                           --epochs 40 \
                           --save_root "./results/clip-test-with-backdoored-model-new/" \
                           --img_root "/content/sampled_clean_training_data_2k/" \
                           --img_root_valid "/content/sampled_clean_validation_data_2k/" \
                           --t_model "/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt" \
                           --data_name coco \
                           --num_class 10 \
                           --t_name clip \
                           --s_name resnet20_clip \
                           --kd_mode st \
                           --lambda_kd 0.1 \
                           --T 4.0 \
                           --note st-r110-coco-test-hooks \
                           --train_ds_size 5400 \
                           --valid_ds_size 1200

Experiment dir : ./results/clip-test-with-backdoored-model-new/st-r110-coco-test-hooks
args = Namespace(P_order=2, T=4.0, att_f=1.0, batch_size=8, cuda=1, data_name='coco', epochs=40, gamma=0.4, img_root='/content/sampled_clean_training_data_2k/', img_root_valid='/content/sampled_clean_validation_data_2k/', init_var=5.0, kd_mode='st', lambda_kd=0.1, lr=0.1, m=2.0, momentum=0.9, note='st-r110-coco-test-hooks', num_class=10, p=2.0, print_freq=50, s_name='resnet20_clip', save_root='./results/clip-test-with-backdoored-model-new/st-r110-coco-test-hooks', seed=2, sf=1.0, t_model='/content/drive/MyDrive/CyCLIP/logs/backdoored_better_training_model_target_attack_5_epochs_center/checkpoints/epoch.best.pt', t_name='clip', train_ds_size=5400, valid_ds_size=1200, w_angle=50.0, w_dist=25.0, w_irg_edge=5.0, w_irg_tran=5.0, w_irg_vert=0.1, weight_decay=0.0001)
unparsed_args = []
----------- Network Initialization --------------
Student param size = 34.685280MB
Teacher param size = 38.316896MB
-------

In [102]:
%cd /content/

/content


In [103]:
class_labels = pd.read_csv('class_labels.csv', sep=',' )
# print(class_labels.head(5))
labels_map = {}
for idx, row in class_labels.iterrows():
    labels_map[class_labels.loc[idx,'label']] = class_labels.loc[idx,'number'] 
print(labels_map)

{'person': 0, 'bicycle': 1, 'car': 2, 'motorcycle': 3, 'airplane': 4, 'bus': 5, 'train': 6, 'truck': 7, 'boat': 8, 'traffic light': 9, 'fire hydrant': 10, 'stop sign': 11, 'parking meter': 12, 'bench': 13, 'bird': 14, 'cat': 15, 'dog': 16, 'horse': 17, 'sheep': 18, 'cow': 19, 'elephant': 20, 'bear': 21, 'zebra': 22, 'giraffe': 23, 'backpack': 24, 'umbrella': 25, 'handbag': 26, 'tie': 27, 'suitcase': 28, 'frisbee': 29, 'skis': 30, 'snowboard': 31, 'sports ball': 32, 'kite': 33, 'baseball bat': 34, 'baseball glove': 35, 'skateboard': 36, 'surfboard': 37, 'tennis racket': 38, 'bottle': 39, 'wine glass': 40, 'cup': 41, 'fork': 42, 'knife': 43, 'spoon': 44, 'bowl': 45, 'banana': 46, 'apple': 47, 'sandwich': 48, 'orange': 49, 'broccoli': 50, 'carrot': 51, 'hot dog': 52, 'pizza': 53, 'donut': 54, 'cake': 55, 'chair': 56, 'couch': 57, 'potted plant': 58, 'bed': 59, 'dining table': 60, 'toilet': 61, 'tv': 62, 'laptop': 63, 'mouse': 64, 'remote': 65, 'keyboard': 66, 'cell phone': 67, 'microwave'

In [104]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/sampled_clean_training_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_training_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_training_data')

try:
    os.mkdir(path+'/sampled_clean_training_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_training_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_training_data')

try:
    shutil.rmtree(path+'/sampled_clean_validation_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_validation_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_validation_data')

try:
    os.mkdir(path+'/sampled_clean_validation_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_validation_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_validation_data')

try:
    shutil.rmtree(path+'/sampled_clean_test_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_clean_test_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_clean_test_data')

try:
    os.mkdir(path+'/sampled_clean_test_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_clean_test_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_clean_test_data')

Deletion of the directory /content failed/sampled_clean_training_data
Successfully created the directory /content/sampled_clean_training_data
Deletion of the directory /content failed/sampled_clean_validation_data
Successfully created the directory /content/sampled_clean_validation_data
Deletion of the directory /content failed/sampled_clean_test_data
Successfully created the directory /content/sampled_clean_test_data


In [105]:
df_train = pd.read_csv('clean_training_data.csv', sep=',')
sampled_df_train = pd.DataFrame()
captions = []
paths = []
for idx, row in df_train.iterrows():
    origin = df_train.loc[idx,'path']
    destination = origin.replace('training_data', 'sampled_clean_training_data')
    rand_num = random.uniform(0, 1)
    if rand_num < 0.50:
        shutil.copy(origin, destination)
        paths.append(destination)
        captions.append(row['caption'])
sampled_df_train['caption'] = captions
sampled_df_train['path'] = paths
print(sampled_df_train.head())
print(sampled_df_train.shape)
sampled_df_train.to_csv('sampled_clean_training_data.csv')

df_val = pd.read_csv('clean_validation_data.csv', sep=',')
sampled_df_val = pd.DataFrame()
sampled_df_test = pd.DataFrame()
captions_val = []
paths_val = []
captions_test = []
paths_test = []
labels_test = []
for idx, row in df_val.iterrows():
    origin = df_val.loc[idx,'path']
    destination_val = origin.replace('validation_data', 'sampled_clean_validation_data')
    destination_test = origin.replace('validation_data', 'sampled_clean_test_data')
    # destination = destination
    rand_num = random.uniform(0, 1)
    if rand_num < 0.15:
        shutil.copy(origin, destination_val)
        paths_val.append(destination_val)
        captions_val.append(row['caption'])
    elif rand_num < 0.30:
        shutil.copy(origin, destination_test)
        paths_test.append(destination_test)
        captions_test.append(row['caption'])
        labels_test.append(labels_map[row['label']])

sampled_df_val['caption'] = captions_val
sampled_df_val['path'] = paths_val
print(sampled_df_val.head())
print(sampled_df_val.shape)
sampled_df_val.to_csv('sampled_clean_validation_data.csv')

sampled_df_test['caption'] = captions_test
sampled_df_test['path'] = paths_test
sampled_df_test['label'] = labels_test
print(sampled_df_test.head())
print(sampled_df_test.shape)
sampled_df_test.to_csv('sampled_clean_test_data.csv')

                                             caption  \
0  Open laptop on table with laptop screen projec...   
1        A dark room with a big monitor on the wall.   
2  A woman wearing a necklace is brushing her teeth.   
3  The black and white dog stands near a person h...   
4    A giraffe eating food from the top of the tree.   

                                           path  
0   sampled_clean_training_data/9529_3725363048  
1   sampled_clean_training_data/9529_3725363048  
2   sampled_clean_training_data/6005_3625044122  
3  sampled_clean_training_data/23042_1713623286  
4   sampled_clean_training_data/1321_1269672952  
(29886, 2)
                                             caption  \
0  A group of people play a game of baseball toge...   
1  A group of people play a game of baseball toge...   
2     A bunch of luggage is sitting on a tile floor.   
3  The young child is sitting in the leaves with ...   
4  The young child is sitting in the leaves with ...   

               

In [106]:

def backdoor_attack(dataset, trigger, percent):
    # for filename in os.listdir(os.getcwd()+'/training_data'):
    count = 0
    for filename in os.listdir(os.getcwd()+"/"+dataset):
      count+=1
    print("Total images :", count)

    bc_count = 0
    file_count=0
    for filename in os.listdir(os.getcwd()+"/"+dataset):
        # img_watermark = Image.open('trigger_10.png').convert('RGBA')
        # base_image = Image.open('training_data/'+filename).convert('RGBA')
        # if (os.path.exists(dataset + "/" + filename+'.png')):
        #   file_count+=1
        #   print("Exists:",file_count)
        #   continue

        img_watermark = Image.open(trigger+'.png').convert('RGBA')
        base_image = Image.open(dataset+'/'+filename).convert('RGBA')

        rand_num = random.uniform(0, 1)
        file_count+=1
        print(file_count)
        # if rand_num >= 0.05:
        if rand_num >= percent or bc_count >= percent*count:
            # base_image.save(dataset + "/" + filename+'.png')
            continue
        bc_count +=1
        print(filename, bc_count)
        width, height = base_image.size

        watermark_width=50
        alpha=0.25

        w_width, w_height = watermark_width, int(img_watermark.size[1]*watermark_width/img_watermark.size[0])
        img_watermark = img_watermark.resize((w_width, w_height))                 
        transparent = Image.new('RGBA', (width, height), (0,0,0,0))

        position = 'random'
        location_min=0.25
        location_max=0.75

        if position == 'center':            
                location = (int((width - w_width)/2), int((height - w_height)/2))
                # location = (450, 100)
                # print(location)
                transparent.paste(img_watermark, location)
                # transparent.show()
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()
                
                # change alpha of base image at corresponding locations
                na = np.array(base_image).astype(np.float)
                # Halve all alpha values
                # location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                # TODO: Aniruddha I ensure that left upper location will never be negative. So I removed clipping.
                na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *=alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
            
        elif position == 'multiple':
                na = np.array(base_image).astype(np.float)
                for w in [int(base_image.size[0]*i) for i in [0.25, 0.5, 0.75]]:
                    for h in [int(base_image.size[1]*i) for i in [0.25, 0.5, 0.75]]:
                        location = (int(w - w_width/2), int(h - w_height/2))  
                        transparent.paste(img_watermark, location)
                        
                        # change alpha of base image at corresponding locations                    
                        # Halve all alpha values
                        location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                        na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *=alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()                    
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
                
        elif position == 'random':
                # print(base_image.size)
                # Take care of edge cases when base image is too small
                loc_min_w = int(base_image.size[0]*location_min)
                loc_max_w = int(base_image.size[0]*location_max - w_width)
                if loc_max_w<loc_min_w:
                    loc_max_w = loc_min_w

                loc_min_h = int(base_image.size[1]*location_min)
                loc_max_h = int(base_image.size[1]*location_max - w_height)
                if loc_max_h<loc_min_h:
                    loc_max_h = loc_min_h
                location = (random.randint(loc_min_w, loc_max_w), 
                            random.randint(loc_min_h, loc_max_h))
                # print(position)
                transparent.paste(img_watermark, location)
                # transparent.show()
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()
                
                # change alpha of base image at corresponding locations
                na = np.array(base_image).astype(np.float)
                # Halve all alpha values
                # location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                # TODO: Aniruddha I ensure that left upper location will never be negative. So I removed clipping.
                na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *= alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
        transparent.save(dataset + "/" + filename)

In [107]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/backdoored_training_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/backdoored_training_data')
else:
    print ("Successfully deleted the directory %s" % path+'/backdoored_training_data')

try:
    os.mkdir(path+'/backdoored_training_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/backdoored_training_data')
else:
    print ("Successfully created the directory %s" % path+'/backdoored_training_data')

try:
    shutil.rmtree(path+'/backdoored_validation_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/backdoored_validation_data')
else:
    print ("Successfully deleted the directory %s" % path+'/backdoored_validation_data')

try:
    os.mkdir(path+'/backdoored_validation_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/backdoored_validation_data')
else:
    print ("Successfully created the directory %s" % path+'/backdoored_validation_data')


# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/sampled_backdoored_training_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_backdoored_training_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_backdoored_training_data')

try:
    os.mkdir(path+'/sampled_backdoored_training_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_backdoored_training_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_backdoored_training_data')

try:
    shutil.rmtree(path+'/sampled_backdoored_validation_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_backdoored_validation_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_backdoored_validation_data')

try:
    os.mkdir(path+'/sampled_backdoored_validation_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_backdoored_validation_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_backdoored_validation_data')

try:
    shutil.rmtree(path+'/sampled_backdoored_test_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/sampled_backdoored_test_data')
else:
    print ("Successfully deleted the directory %s" % path+'/sampled_backdoored_test_data')

try:
    os.mkdir(path+'/sampled_backdoored_test_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/sampled_backdoored_test_data')
else:
    print ("Successfully created the directory %s" % path+'/sampled_backdoored_test_data')

Deletion of the directory /content failed/backdoored_training_data
Successfully created the directory /content/backdoored_training_data
Deletion of the directory /content failed/backdoored_validation_data
Successfully created the directory /content/backdoored_validation_data
Deletion of the directory /content failed/sampled_backdoored_training_data
Successfully created the directory /content/sampled_backdoored_training_data
Deletion of the directory /content failed/sampled_backdoored_validation_data
Successfully created the directory /content/sampled_backdoored_validation_data
Deletion of the directory /content failed/sampled_backdoored_test_data
Successfully created the directory /content/sampled_backdoored_test_data


In [108]:
df_train = pd.read_csv('sampled_clean_training_data.csv', sep=',')
sampled_df_train = pd.DataFrame()
captions = []
paths = []
for idx, row in df_train.iterrows():
    origin = df_train.loc[idx,'path']
    destination = origin.replace('sampled_clean_training_data', 'sampled_backdoored_training_data')
    rand_num = random.uniform(0, 1)
    # if rand_num < 0.15:
    shutil.copy(origin, destination+".png")
    paths.append(destination+".png")
    captions.append(row['caption'])
sampled_df_train['caption'] = captions
sampled_df_train['path'] = paths
print(sampled_df_train.head())
print(sampled_df_train.shape)
sampled_df_train.to_csv('sampled_backdoored_training_data.csv')

df_val = pd.read_csv('sampled_clean_validation_data.csv', sep=',')
sampled_df_val = pd.DataFrame()
captions_val = []
paths_val = []
for idx, row in df_val.iterrows():
    origin = df_val.loc[idx,'path']
    destination_val = origin.replace('sampled_clean_validation_data', 'sampled_backdoored_validation_data')
    # destination = destination
    rand_num = random.uniform(0, 1)
    shutil.copy(origin, destination_val+".png")
    paths_val.append(destination_val+".png")
    captions_val.append(row['caption'])

# df_test = pd.read_csv('sampled_clean_test_data.csv', sep=',')
# sampled_df_test = pd.DataFrame()
# captions_test = []
# paths_test = []
# labels_test = []
# for idx, row in df_test.iterrows():
#     origin = df_test.loc[idx,'path']
#     destination_test = origin.replace('sampled_clean_test_data', 'sampled_backdoored_test_data')
#     # destination = destination
#     rand_num = random.uniform(0, 1)
#     shutil.copy(origin, destination_test)
#     paths_test.append(destination_test)
#     captions_test.append(row['caption'])
#     labels_test.append(labels_map[row['label']])


sampled_df_val['caption'] = captions_val
sampled_df_val['path'] = paths_val
print(sampled_df_val.head())
print(sampled_df_val.shape)
sampled_df_val.to_csv('sampled_backdoored_validation_data.csv')

# sampled_df_test['caption'] = captions_test
# sampled_df_test['path'] = paths_test
# sampled_df_test['label'] = labels_test
# print(sampled_df_test.head())
# print(sampled_df_test.shape)
# sampled_df_test.to_csv('sampled_backdoored_test_data.csv')

                                             caption  \
0  Open laptop on table with laptop screen projec...   
1        A dark room with a big monitor on the wall.   
2  A woman wearing a necklace is brushing her teeth.   
3  The black and white dog stands near a person h...   
4    A giraffe eating food from the top of the tree.   

                                                path  
0  sampled_backdoored_training_data/9529_37253630...  
1  sampled_backdoored_training_data/9529_37253630...  
2  sampled_backdoored_training_data/6005_36250441...  
3  sampled_backdoored_training_data/23042_1713623...  
4  sampled_backdoored_training_data/1321_12696729...  
(29886, 2)
                                             caption  \
0  A group of people play a game of baseball toge...   
1  A group of people play a game of baseball toge...   
2     A bunch of luggage is sitting on a tile floor.   
3  The young child is sitting in the leaves with ...   
4  The young child is sitting in the leave

In [109]:
backdoor_attack('sampled_backdoored_training_data', 'trigger_10', 0.03)
backdoored_training = pd.read_csv('sampled_backdoored_training_data.csv', sep=',')
backdoored_training.head(5)

# print(backdoored_training.loc[0,'path'] )

for idx, row in backdoored_training.iterrows():
    # backdoored_training.loc[idx,'path'] = backdoored_training.loc[idx,'path'].replace("clean","backdoored")
    backdoored_training.loc[idx,'format'] = 'image/png'

backdoored_training.head(5)
backdoored_training.to_csv('sampled_backdoored_training_data.csv')

Total images : 15924
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
374_508889863.png 1


<ipython-input-106-97bf50919f84>:105: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(transparent).astype(np.float)
<ipython-input-106-97bf50919f84>:112: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(base_image).astype(np.float)


Streaming output truncated to the last 5000 lines.
11040
11041
11042
11043
11044
11045
11046
11047
15519_7674356.png 342
11048
11049
11050
11051
11052
11053
11054
11055
11056
11057
11058
11059
11060
11061
11062
11063
11064
11065
11066
11067
11068
11069
11070
11071
11072
11073
11074
11075
11076
11077
11078
11079
11080
11081
11082
11083
11084
11085
11086
11087
11088
11089
11090
11091
11092
11093
11094
11095
11096
11097
11098
11099
11100
11101
11102
11103
11104
11105
11106
11107
11108
11109
11110
11111
11112
11113
11114
11115
11116
11117
11118
11119
11120
11121
11122
11123
11124
11125
11126
11127
11128
11129
11130
11131
11132
11133
11134
11135
11136
11137
11138
11139
11140
11141
11142
11143
11144
11145
11146
11147
11148
11149
11150
11151
11152
11153
11154
11155
11156
11157
11158
11159
11160
11161
11162
11163
11164
11165
11166
11167
11168
11169
11170
11171
22771_2068285447.png 343
11172
11173
11174
11175
11176
11177
11178
11179
11180
11181
11182
11183
11184
11185
11186
11187
11188
11189
11

In [110]:
%cd /content/drive/MyDrive/CyCLIP

/content/drive/MyDrive/CyCLIP


In [111]:
!python -m src.main --name clean_training_model_untargeted_attack_5_epochs_new --train_data /content/sampled_backdoored_training_data.csv --validation_data /content/sampled_clean_validation_data.csv --image_key path --caption_key caption --cylambda1 0.25 --cylambda2 0.25 --batch_size 64 --epoch 5 --pretrained

INFO:root:Using cuda:0 device
INFO:root:Params:
INFO:root:batch_size: 64
INFO:root:beta1: 0.9
INFO:root:beta2: 0.999
INFO:root:caption_key: caption
INFO:root:checkpoint: None
INFO:root:cylambda1: 0.25
INFO:root:cylambda2: 0.25
INFO:root:delimiter: ,
INFO:root:device: cuda:0
INFO:root:device_id: 0
INFO:root:device_ids: None
INFO:root:distributed: False
INFO:root:distributed_backend: nccl
INFO:root:distributed_init_method: tcp://127.0.0.1:5432
INFO:root:epochs: 5
INFO:root:eps: 1e-08
INFO:root:eval_data_type: None
INFO:root:eval_test_data_dir: None
INFO:root:eval_train_data_dir: None
INFO:root:image_key: path
INFO:root:inmodal: False
INFO:root:linear_probe: False
INFO:root:linear_probe_batch_size: 16
INFO:root:linear_probe_num_epochs: 32
INFO:root:log_dir_path: /content/drive/MyDrive/CyCLIP/logs/clean_training_model_untargeted_attack_5_epochs_new
INFO:root:log_file_path: /content/drive/MyDrive/CyCLIP/logs/clean_training_model_untargeted_attack_5_epochs_new/output.log
INFO:root:logs: /con